#### *If you are clear with basics you can skip to code below to see how to create and freeze a 1 node graph (section 2.Hands On)*

## 1.Basics

**General concepts**

As per google, Tensorflow is defined as 
>TensorFlow™ is an open source software library for ***numerical computation*** using data flow graphs. ***Nodes in the graph*** represent mathematical operations, while the ***graph edges*** represent the multidimensional ***data arrays (tensors)*** communicated between them.

Imagine, a sophistcated AI framework extensively used by google in all there products defined in two simple lines. Beauty of the above statement is, this infact is the essence of all the Deep Learning happening today. And this defination is applicable to every Deep Learning framework we have.

Lets take a typical neural network (basically a graph) and understand the above two lines

<img src="../images/t1_1_basic_nn_color.jpeg" alt="Drawing" style="width: 600px;"/>

<br></br>

In any kind of deep learning we create a graph (called as a model in popular language) in which a

**NODE** -> is a mathematical operation like add, subtract, multiply or some other operation performed on matrices recieved as input. Necessarily not 2 dimensional matrices but can be multi-dimensional matrices called tensors. Now you know what tensor->flow actually means. **(ask yourself add, subtract or multiply the input with what?** will answer that shortly **)**

For example, Node A1 in Computation layer 1 takes input from all the Input nodes (1,2 and 3) and does the computation on each input.

**Graph Edge** -> in a deep learning model basically means the output of a node will be passed to all the nodes it is connect to in the next layer. In above diagram from example, Node A1 will pass on its output to the nodes in next layer it is connected to ie Node B1, B2, B3 and B4.


Till now we were talking about operations done by nodes but what about data we input and how is it handled. Lets see it next.


**Lets bring in some key Tensorflow concepts**

**1.Holders** -> These are like variables in language of computer programming used to hold values. For us there are three type of value holders we should be concerned. 
- **placeholders** -> these are the entry points from where input values are fed into graphs(neural network). In the diagram above Input nodes (1,2 and 3) are actually placeholders to feed data into network.

- **variables** -> these are fixed dimension holders whose values can change.

- **constants** -> these are fixed dimension holders whose values can't be changed once after assigning them some value.

**2.Graph** -> is a collection of nodes which take input from placeholders, process them in nodes and give output. 

**3.Session** -> To operate on a graph we need to create a session for that. This ensures only 1 program is able to run that graph so that learnings are consistent.
<br></br>

**Lets see the last important concept where all the magic is happening, the *learning* in deep learning.**

While explaining nodes above, I told you that nodes take in input from previous layer, do a mathematical operation say multiply and pass on the output to next layer. But multiply with what ? 

Here comes the model **parameters** in picture.

Every computation node in a neural network have some parameters(also called weights) which are used in doing the computation assigned to that node. These weights are **tensorflow variable(another matrix or more appropriately a tensor)** holding some magic values which is used in computation done on the input and yield the output.

May sound complicated, for now just assume that every node holds some magic value to do its task and generate output for next layer from a given input. 

Also in next tutorial I'll introduce the concept of training and iterations and how a model learns to make predictions. There you will get more clarity on what these weights are and how are their magic values assigned to them.
<br></br>

**Summarizing what we have read till now**

In deep learning
- We create graph like computation structure popularly called as model.
- A graph consist of computation nodes and some parameters in those nodes with magic values in them.
- In tensorflow we have a special variable called placeholder to feed input into graph.
- Once we create a graph, we create a tensorflow session to operate on that graph.

## 2.Hands On

Before we start with code lets outline what are we going to do.

- We will create a single node Neural network which simply adds two input values.
- Create a tensorflow session, initialize all the variables we define and run the session.
- Save the model onto harddisk as a ***.pbtxt*** file. (more details of this file ahead).
- Freeze the model and save it as a ***.pb*** file which we can deploy on android device.
- Copy the saved file in the assets directory of TFAndroid android project.
- Test your first tensorflow model on an android device.

<img src="../images/t1_2_single_node_network.jpeg" alt="Drawing" style="width: 400px;"/>


In [ ]:
import tensorflow as tf
print("TensorFlow version: " + tf.__version__)

In [ ]:
TUTORIAL_NAME = 'Tutorial1'
MODEL_NAME = 'basicTFonAndroid'
SAVED_MODEL_PATH = '../' + TUTORIAL_NAME+'_Saved_model/'

In [ ]:
A = tf.placeholder(tf.float32, shape=[1], name='some_nameA') # input a
B = tf.placeholder(tf.float32, shape=[1], name='some_nameB') # input b

# A tensorflow variable not used anywhere in this model 
W = tf.Variable(tf.zeros(shape=[1]), dtype=tf.float32, name='some_nameW') # weights

# sum of two matrices element-wise, in our case two numbers in 
AB = tf.add(A, B, name='some_nameAB')

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
sess.run(AB,{A:[2],B:[4]})

In [ ]:
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '_withWeightW_asVariable' + '.pbtxt')

In [ ]:
_W = W.eval(sess)
print(_W)

In [ ]:
A2 = tf.placeholder(tf.float32, shape=[1], name='modelInputA') # input a
B2 = tf.placeholder(tf.float32, shape=[1], name='modelInputB') # input b

# A tensorflow variable(not used anywhere in this model)
W2 = tf.constant(_W, name='modelWeightW') # weights

# sum of two matrices element-wise, in our case two numbers in 
AB2 = tf.add(A2, B2, name='modelOutputAB')

In [ ]:
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pbtxt')
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pb', as_text=False)